In [1]:
# Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
pd.set_option('display.max_rows',50)

In [2]:
# Create a path to the csv and read it into a Pandas DataFrame
#csv_path = "Assessor_Historical_Secured_Property_Tax_Rolls.csv"
csv_path = "../Home Prices/combine_updated.csv"
rent_path = "../Data/yearly_rent.csv"
home_df = pd.read_csv(csv_path, encoding='utf8', engine='python')
rent_df = pd.read_csv(rent_path, encoding='utf8', engine='python')

In [3]:
#Changing from columns to rows
my_col_names = list(rent_df.columns.values)[2:9]
melted_df = pd.melt(rent_df,id_vars=["Neighborhood"],
                    value_vars=my_col_names, var_name="Year",value_name="Yearly Rent Price")

In [4]:
melted_df.head()

,Neighborhood,Year,Yearly Rent Price
0,Bayview,2011,30723
1,Bernal Heights,2011,34471
2,Buena Vista,2011,42407
3,Corona Heights,2011,41051
4,Cow Hollow,2011,52856


In [5]:
#merging two data frames
rent_df2 = pd.merge(melted_df,rent_df.loc[:,["Neighborhood","City","Lat","Lng"]])

In [6]:
#rename columns to merge on same names
#changing the type of column
home_df = home_df.rename(columns={'Assessor Neighborhood': 'Neighborhood', 
                                  'Closed Roll Year': 'Year'})
rent_df2["Year"] = rent_df2["Year"].astype(int)

In [7]:
home_df = home_df.drop(['Unnamed: 0'], axis=1)
home_df = home_df.drop(['Pct Change'], axis=1)

In [8]:
#merging
merged_csv = pd.merge(home_df,rent_df2, 
                  how='inner',
                  left_on=["Neighborhood","Year"],
                  right_on=["Neighborhood","Year"])

In [9]:
merged_csv.head()

,Neighborhood,Year,Assessed Land Value,Yearly Rent Price,City,Lat,Lng
0,Bayview,2011,175691.806133,30723,San Francisco,37.730416,-122.384424
1,Bayview,2012,177818.914467,28821,San Francisco,37.730416,-122.384424
2,Bayview,2013,182203.741400,30433,San Francisco,37.730416,-122.384424
3,Bayview,2014,185691.451367,35338,San Francisco,37.730416,-122.384424
4,Bayview,2015,198571.039233,42870,San Francisco,37.730416,-122.384424


In [10]:
merged_gb = merged_csv.groupby("Neighborhood").mean()

In [11]:
merged_gb = merged_gb.drop(['Year'], axis=1)

In [12]:
merged_gb.head()

,Assessed Land Value,Yearly Rent Price,Lat,Lng
Neighborhood,,,,
Bayview,190512.858289,35644.333333,37.730416,-122.384424
Bernal Heights,268491.454817,43599.666667,37.738950,-122.415201
Buena Vista,487987.841867,53112.333333,37.806504,-122.420778
Corona Heights,394108.123417,52034.000000,37.761812,-122.443185
Cow Hollow,614482.066967,63147.833333,37.798033,-122.439646


In [13]:
#combined.to_csv('combine.csv', sep=',', encoding='utf-8')
merged_csv.to_csv('../Data/hp_and_rent.csv', sep=',', encoding='utf-8')
merged_gb.to_csv('../Data/hp_and_rent_grouped.csv', sep=',', encoding='utf-8')